<span style="color:lightgreen; font-size:30px">**PG301 - Geoestadística**</span>
***
<span style="color:gold; font-size:30px">**Variograma 1D**</span>
***

<span style="font-size:20px"> **Autor: Kevin Alexander Gómez** </span>

<span style="font-size:16px"> **Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)** </span>

***

Bienvenido al curso PG301 - Geoestadística!!!

Vamos a revisar las bases de la <span style="color:gold">geoestadística</span> usando ejemplos en Python.\
Es necesario que tengas un conocimiento previo en programación con Python, estadística y geología general.

<span style="color:lightgreen"> Este notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología. </span>

En el siguiente índice, encontrarás los temas que componen este notebook:

## **Índice**
***
- [¿Qué es `gamma`?](#parte-1)
- [Procesamiento de datos](#parte-2)
- [Creación del variograma experimental](#parte-3)
- [Intervalos de espaciamiento (Binning)](#parte-4)
- [Ajuste del modelo de variograma (Fitting)](#parte-5)
***

Antes de empezar tu camino en programación geológica...\
Recuerda que puedes ejecutar un bloque de código usando `Shift` + `Enter`:

In [ ]:
2 + 2

Si por error haces doble clic sobre un bloque de texto (como el que estás leyendo ahora mismo), puedes arreglarlo usando también `Shift` + `Enter`.

***

<a id="parte-1"></a>

### <span style="color:lightgreen">**¿Qué es `gamma`?**</span>
***    

El módulo de geoestadística <span style="color:gold">gamma</span> ha sido implementado con la finalidad de usar herramientas geoestadísticas (como la función variograma) para la visualización interactiva y el aprendizaje dentro de un notebook de Jupyter.

Por el momento, `gamma` solo cuenta con el variograma 1D, llamado a través de la función `Variogram`.\
También existen modelos de variogramas que podemos usar como `linear`, `exponential`, `spherical` o `gaussian`.

La funcionalidad básica de `Variogram` se describe con los siguientes pasos:
- Empezamos cargando los datos correspondientes a la variable de interés `values` y las distancias de separación `coords`.
- Luego, usamos `Variogram` para calcular los intervalos de separación o también llamados bins del variograma experimental. La distancia de separación usada para el variograma experimental se puede ajustar con el método `selectLagDist`.
- Una vez que se ha determinado la distancia de separación, se proceden a ajustar los modelos de variograma mencionados anteriormente a través del método `fitting`.
- Este método provee de una interfaz para ajustar manualmente los valores de c0 (pepita), c (sill parcial) y a (alcance).
- Una vez ajustado y seleccionado el modelo de variograma, podemos exportar sus parámetros en un diccionario usando el atributo `model`.

***

<a id="parte-2"></a>

### <span style="color:lightgreen">**Procesamiento de datos**</span>
***    

Empezamos importando `pandas` para cargar el archivo `assay.csv`.\
Seleccionamos las leyes y coordenadas de uno de los taladros para modelar el variograma 1D.
> Nota: el módulo `gamma` se encuentra ubicado dentro de la carpeta `files`.

In [ ]:
import pandas as pd
import numpy as np
from files.gamma import Variogram

In [ ]:
data = pd.read_csv("files/assay.csv")[["ID", "FROM", "TO", "CU_pct"]]
data = data[data["ID"] == "DH010"].drop(columns=["ID"]).reset_index(drop=True)

Para crear la columna de coordenadas, calcularemos el centroide para cada tramo del sondaje:

In [ ]:
data["coords"] = (data["FROM"] + data["TO"]) / 2

Miramos la tabla un momento usado `head`:

In [ ]:
data.head()

Ya tenemos una columna con los centroides de cada tramo del sondaje.\
Ahora, seleccionaremos los valores y coordenadas para almacenarlos en las variables `values` y `coords`:

In [ ]:
values = data["CU_pct"].copy().values
coords = data["coords"].copy().values

¿Cuántos datos tenemos en total?

In [ ]:
print(f"Cantidad total de datos: {len(values)}.")

Podemos observar que tenemos 305 tramos en el sondaje DH010.\
A continuación, usaremos `values` y `coords` para inicializar `Variogram` y calcular el variograma experimental.

***

<a id="parte-3"></a>

### <span style="color:lightgreen">**Creación del variograma experimental**</span>
***

Usaremos la función `Variogram` para crear el objeto que representará el variograma dentro de Python.\
Asignaremos el variograma a la variable `var1d`:

In [ ]:
var1d = Variogram(values=values, coords=coords)

Si llamamos la variable `var1d`, obtendremos una descripción del contenido actual: 

In [ ]:
print(var1d)

***

<a id="parte-4"></a>

### <span style="color:lightgreen">**Intervalos de espaciamiento (Binning)**</span>
***

Las posiciones de las muestras no siempre son exactas, o están distribuidas a distancias exactas.\
Por lo tanto, existe una necesidad de agrupar los pares calculados (de distancia y diferencia cuadrática) de acuerdo al espaciamiento que tienen entre sí (**lag classes**).

El objetivo de esta parte es el de ajustar el intervalo de espaciamiento entre muestras, con la finalidad de facilitar y asegurar que el modelamiento del variograma sea correcto.

Si usamos el atributo `lagDist`, observaremos la distancia de espaciamiento seleccionada automáticamente por el programa:

In [ ]:
var1d.lagDist

Podemos modificar manualmente esta distancia de espaciamiento usando el método `selectLagDist`, el programa recalculará los intervalos asociados a esta distancia.
> Debemos seleccionar en el widget la distancia de 4.00 metros.\
> El gráfico muestra las primeras distancias de separación entre muestras a lo largo de una recta numérica.\
> Es posible distinguir agrupamientos (clusters) a lo largo de dicha recta.\
> Puedes deslizar el valor `h0` para cambiar el intervalo de espaciamiento entre muestras.\
> El botón verde `Asignar lagDist` guarda el intervalo seleccionado para el variograma.

#### <span style="color:gold">Dentro del siguiente widget, selecciona una distancia de espaciamiento de 4.00 metros:</span>

In [ ]:
# Widget de selección de intervalo de espaciamiento
var1d.selectLagDist()

Verificamos que la distancia haya sido modificada:

In [ ]:
print(var1d)

Notamos que el intervalo de espaciamiento mínimo entre muestras ahora es de **4 metros**.\
Una vez seleccionada esta distancia, podemos pasar al ajuste de un modelo de variograma.
***

<a id="parte-5"></a>

### <span style="color:lightgreen">**Ajuste del modelo de variograma (Fitting)**</span>
***

Una vez hemos seleccionado un intervalo de espaciamiento adecuado, podemos graficar el **variograma experimental** y usarlo para ajustar un modelo de variograma que puede ser lineal, exponencial, esférico o gaussiano.

Usaremos el método `fitting` para generar los datos de ajuste de cada tipo de modelo y, a través de una visualización interactiva, podemos elegir aquel modelo que se ajuste mejor al variograma experimental:
> Desliza el valor `n_points` para cambiar el número de puntos de ajuste, y `model` para elegir el modelo.\
> El parámetro `Autoajuste` activa y desactiva los parámetros de ajuste manual del variograma.\
> Los parámetros `Pepita (C0)`, `Sill Parcial (C)` y `Alcance (a)` son usados para ajustar manualmente el modelo.\
> El botón `Guardar parámetros` asigna los parámetros de ajuste del modelo en el atributo `model`.

In [ ]:
var1d.fitting()

Notamos que el variograma empieza a tener un comportamiento diferente a partir de los 200 metros, por lo que seleccionaremos un modelo que se ajuste hasta esa distancia.

Notamos también que el alcance de nuestro modelo será de aprox. 100 metros.

El modelo seleccionado cumplirá con las siguientes características:
- Modelo: `spherical`
- Puntos: `52`
- Pepita (C0): `0.111`
- Sill Parcial (C): `0.149`
- Alcance (a): `100`

Guardamos los parámetros y mostramos la descripción del variograma:

In [ ]:
print(var1d)

El atributo `model` contiene los parámetros seleccionados para el modelo de variograma.

In [ ]:
var1d.model

Estos resultados pueden ser exportados como diccionario y ser usados en otros flujos de trabajo.

***